# Training Job in Internet-free Mode

If you want to isolate your training data and training container from the rest of the Internet, then you should create the training job in a private subnet. A private subnet is a subnet in your VPC without a route to Internet Gateway. This means, by default, no inbounding calls to your container from the Internet is possible and your container cannot make outbounding calls to the Internet. If you need the training container to access your S3 resource, you need to **explicitly** add a VPC endpoint and attach it to the route table of your private subnet to allow traffic to your data in S3 bucket. 

In this notebook, we will walk through an example of creating such a training job. We will

- Build a simple training image
- Set up a VPC
- Set up a private subnet in the VPC
- Set up a security group in the VPC
- Create a training job in your private subnet && security group and watch it to fail (because it cannot access your S3 resource)
- Add a VPC endpoint to allow traffic to S3
- Create another training job in your private subnet and watch it to succeeed 

If you are not familiar with VPC security configuration, the following materials can help you
- [Security in Amazon Virtual Private Cloud](https://docs.aws.amazon.com/vpc/latest/userguide/security.html)
- [Training and Inference Containers in Internet-Free Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/mkt-algo-model-internet-free.html)

It's okay if you don't understand everything from the official docs above, the code samples you will see in this notebook will help you grasp those concepts. 

In [1]:
# import libraries
import boto3
import pprint
import datetime
import time

pp = pprint.PrettyPrinter(indent=1)

## Build a training image

We follow the same procedure for building a training image as in [this notebook](https://github.com/hsl89/amazon-sagemaker-examples/blob/sagemaker-fundamentals/sagemaker-fundamentals/create-training-job/create-training-job.ipynb). We will refer to this image as `example-image`. Please go through that notebook if you are not familiar with `CreateTrainingJob` API.

In [3]:
# create a repo in your ECR 

ecr = boto3.client('ecr')

try:
    # The repository might already exist
    # in your ECR
    cr_res = ecr.create_repository(
        repositoryName='example-image')
    pp.pprint(cr_res)
except Exception as e:
    print(e)

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'example-image' already exists in the registry with id '688520471316'


In [4]:
%%sh
# build the image
cd container/

# tag it as example-image:latest
docker build -t example-image:latest .
    
# test the container
python local_test/test_container.py

account=$(aws sts get-caller-identity --query Account | sed -e 's/^"//' -e 's/"$//')
region=$(aws configure get region)
ecr_account=${account}.dkr.ecr.${region}.amazonaws.com

# Give docker your ECR login password
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr_account

# Fullname of the repo
fullname=$ecr_account/example-image:latest

# Tag the image with the fullname
docker tag example-image:latest $fullname

# Push to ECR
docker push $fullname

Sending build context to Docker daemon   16.9kB
Step 1/4 : FROM continuumio/miniconda:latest
 ---> b8ea69b5c41c
Step 2/4 : RUN mkdir -p /opt/ml
 ---> Using cache
 ---> a170cc3fed03
Step 3/4 : COPY train.py /usr/bin/train
 ---> Using cache
 ---> 4e19c1cb2076
Step 4/4 : RUN chmod +x /usr/bin/train
 ---> Using cache
 ---> 38609e5aaa0d
Successfully built 38609e5aaa0d
Successfully tagged example-image:latest
== Files in train channel ==
== Files in the test channel ==
== Saving model checkpoint ==
== training completed ==

Login Succeeded
The push refers to repository [688520471316.dkr.ecr.us-west-2.amazonaws.com/example-image]
967579d07803: Preparing
5c8c2d1bcfe6: Preparing
88674bdc7fd9: Preparing
78db50750faa: Preparing
805309d6b0e2: Preparing
2db44bce66cd: Preparing
2db44bce66cd: Waiting
805309d6b0e2: Layer already exists
967579d07803: Layer already exists
5c8c2d1bcfe6: Layer already exists
88674bdc7fd9: Layer already exists
78db50750faa: Layer already exists
2db44bce66cd: Layer already 

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Create a VPC

You can think of Amazon VPC as the traditional network in a data center in the cloud. 

The following are the key concepts for VPCs: 
* Virtual private cloud (VPC) — A virtual network dedicated to your AWS account.
* Subnet — A range of IP addresses in your VPC.
* Route table — A set of rules, called routes, that are used to determine where network traffic is directed.
* Internet gateway — A gateway that you attach to your VPC to enable communication between resources in your VPC and the internet.
* VPC endpoint — Enables you to privately connect your VPC to supported AWS services and VPC endpoint services powered by PrivateLink without requiring an internet gateway, NAT device, VPN connection, or AWS Direct Connect connection. Instances in your VPC do not require public IP addresses to communicate with resources in the service. Traffic between your VPC and the other service does not leave the Amazon network. For more information, see AWS PrivateLink and VPC endpoints.
* CIDR block —Classless Inter-Domain Routing. An internet protocol address allocation and route aggregation methodology. For more information, see [Classless Inter-Domain Routing](https://en.wikipedia.org/wiki/Classless_Inter-Domain_Routing#CIDR_notation) in Wikipedia.

All of these concepts are explained in the [official docs](https://docs.aws.amazon.com/vpc/latest/userguide/what-is-amazon-vpc.html). 


In [5]:
# Create a VPC in your default region

ec2 = boto3.client('ec2')

vpc_res = ec2.create_vpc(
    CidrBlock='10.0.0.0/20', # 2^(32 - 20) = 4906 private ipv4 addrs
    AmazonProvidedIpv6CidrBlock=False,
    DryRun=False,
    TagSpecifications=[
        {
            'ResourceType': 'vpc', 
            'Tags':[
                {
                    'Key': 'Name',
                    'Value': 'hello-world'
                },
            ]
        },
    ]
)

pp.pprint(vpc_res)

{'ResponseMetadata': {'HTTPHeaders': {'cache-control': 'no-cache, no-store',
                                      'content-length': '1054',
                                      'content-type': 'text/xml;charset=UTF-8',
                                      'date': 'Fri, 19 Mar 2021 01:15:46 GMT',
                                      'server': 'AmazonEC2',
                                      'strict-transport-security': 'max-age=31536000; '
                                                                   'includeSubDomains',
                                      'x-amzn-requestid': '9fd072ec-01be-4ecb-b9e3-2e9a317d3e67'},
                      'HTTPStatusCode': 200,
                      'RequestId': '9fd072ec-01be-4ecb-b9e3-2e9a317d3e67',
                      'RetryAttempts': 0},
 'Vpc': {'CidrBlock': '10.0.0.0/20',
         'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-0b6e7b82620de322c',
                                      'CidrBlock': '10.0.0.0/20',
       

In [6]:
# inspect this VPC in details

vpc_des = ec2.describe_vpcs(
    VpcIds=[vpc_res['Vpc']['VpcId']]
    )
pp.pprint(vpc_des['Vpcs'])

[{'CidrBlock': '10.0.0.0/20',
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-0b6e7b82620de322c',
                               'CidrBlock': '10.0.0.0/20',
                               'CidrBlockState': {'State': 'associated'}}],
  'DhcpOptionsId': 'dopt-8b1f91ef',
  'InstanceTenancy': 'default',
  'IsDefault': False,
  'OwnerId': '688520471316',
  'State': 'available',
  'Tags': [{'Key': 'Name', 'Value': 'hello-world'}],
  'VpcId': 'vpc-0b52a6097c86ebdc3'}]


In [7]:
# create subnet and associate it with route table

def get_first_availability_zone():
    region_name = boto3.Session().region_name
    avz_res = ec2.describe_availability_zones(
        Filters=[
            {
                "Name": "region-name",
                "Values": [region_name]
            }
        ],
        AllAvailabilityZones=True,
    )
    
    for az in avz_res['AvailabilityZones']:
        if az['ZoneType']=='availability-zone':
            return az
    else:
        return None
    
def create_subnet(vpc_id, cidr_block, dry_run):
    """Create a subnet in the first availability zone in your current region"""
    az = get_first_availability_zone()
    if az is not None:
        subnet_res = ec2.create_subnet(
            AvailabilityZone = az['ZoneName'], #'us-west-2a',
            VpcId = vpc_id,                    # vpc_res['Vpc']['VpcId'],
            CidrBlock= cidr_block,             #'100.68.0.18/18',
            DryRun=dry_run                     # True,
        )
        return subnet_res
    else:
        raise "No availability zone"
        
sn_res = create_subnet(
    vpc_id=vpc_res['Vpc']['VpcId'],
    cidr_block='10.0.0.0/28', # I want 2 ^ (32 - 28) private ipv4 in this subnet
    dry_run=False)

pp.pprint(sn_res)

{'ResponseMetadata': {'HTTPHeaders': {'cache-control': 'no-cache, no-store',
                                      'content-length': '927',
                                      'content-type': 'text/xml;charset=UTF-8',
                                      'date': 'Fri, 19 Mar 2021 01:22:03 GMT',
                                      'server': 'AmazonEC2',
                                      'strict-transport-security': 'max-age=31536000; '
                                                                   'includeSubDomains',
                                      'x-amzn-requestid': 'aa405e89-96cc-4a17-9552-9fd19d1a2429'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'aa405e89-96cc-4a17-9552-9fd19d1a2429',
                      'RetryAttempts': 0},
 'Subnet': {'AssignIpv6AddressOnCreation': False,
            'AvailabilityZone': 'us-west-2a',
            'AvailabilityZoneId': 'usw2-az2',
            'AvailableIpAddressCount': 11,
            'Cidr

In [10]:
# create a security group

sg_res = ec2.create_security_group(
    Description='security group for SageMaker instances',
    GroupName='sagemaker-private',
    VpcId=vpc_res['Vpc']['VpcId'],
    TagSpecifications=[
        {
            "ResourceType": "security-group",
            "Tags" : [
                {   
                    "Key": "Service", # Tag the sec gp by service, this can be used to filter sec gps
                    "Value": "SageMaker" 
                }
            ]
        }
    ]

)

pp.pprint(sg_res)

{'GroupId': 'sg-023956e2fa664f5dd',
 'ResponseMetadata': {'HTTPHeaders': {'cache-control': 'no-cache, no-store',
                                      'content-length': '409',
                                      'content-type': 'text/xml;charset=UTF-8',
                                      'date': 'Fri, 19 Mar 2021 01:22:38 GMT',
                                      'server': 'AmazonEC2',
                                      'strict-transport-security': 'max-age=31536000; '
                                                                   'includeSubDomains',
                                      'x-amzn-requestid': '7a29e519-64c5-42f2-8f4b-b5a9a697b6c5'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7a29e519-64c5-42f2-8f4b-b5a9a697b6c5',
                      'RetryAttempts': 0},
 'Tags': [{'Key': 'Service', 'Value': 'SageMaker'}]}


In [11]:
# inspect the security group in detail

ec2.describe_security_groups(
    GroupIds=[
        sg_res['GroupId']
    ]
)

{'SecurityGroups': [{'Description': 'security group for SageMaker instances',
   'GroupName': 'sagemaker-private',
   'IpPermissions': [],
   'OwnerId': '688520471316',
   'GroupId': 'sg-023956e2fa664f5dd',
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'Tags': [{'Key': 'Service', 'Value': 'SageMaker'}],
   'VpcId': 'vpc-0b52a6097c86ebdc3'}],
 'ResponseMetadata': {'RequestId': '680b4acb-13b2-4de0-8234-e9b6ef93ce2c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '680b4acb-13b2-4de0-8234-e9b6ef93ce2c',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1233',
   'date': 'Fri, 19 Mar 2021 01:22:40 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}